https://github.com/facebookarchive/python-instagram

https://github.com/LevPasha/Instagram-API-python/blob/master/InstagramAPI/InstagramAPI.py


In [ ]:
#!pip install InstagramAPI

In [2]:
from InstagramAPI import InstagramAPI
import time
from datetime import datetime
import db_reacao

Fail to import moviepy. Need only for Video upload.


In [3]:
api = InstagramAPI("vitor.ghiotti@gmail.com", "Deb1Mis2@@")
if (api.login()):
    print("")
else:
    print("Can't login!")

Request return 429 error!
{'message': 'Please wait a few minutes before you try again.', 'status': 'fail'}
Login success!




# PESQUISA PARA ENTENDER USERID, HASHTAGS E LOCATIONS

In [9]:
query = {}
api.fbUserSearch("casariobotequim") 
query = api.LastJson

In [ ]:
print(query.keys())
print("Users".center(100,"*"))
print(query.get("users"))
print("places".center(100,"*"))
print(query.get("places"))
print("hashtags".center(100,"*"))
print(query.get("hashtags"))
print("has_more".center(100,"*"))
print(query.get("has_more"))


# INICIALIZAÇÃO DE VARIAVEIS

In [13]:
query_time_ini = datetime.strptime("2018-01-01", '%Y-%m-%d')
query_time_end = datetime.strptime("2018-04-02", '%Y-%m-%d')
query_start_collect = datetime.strptime("2018-01-01", '%Y-%m-%d')
usernameId = "1229681384"
listHashTags = ["casariobotequim","casariobotequimmais","casariobotequim💣💥"]
listLocations = ["257354853"]
#listLocations = ["257354853","1008118425"]
#hashTags = ["casariobotequim"]

# DEFINIÇÃO DE FUNÇÕES

In [32]:
def get_comments_by_usernameId(usernameId):
    more_available = True
    max_id = ''
    userFeed = {}
    
    api.getUserFeed(usernameId=usernameId,maxid=max_id)
    userFeed = api.LastJson

    while more_available:
        for itens in range(0,userFeed["num_results"]):
            if userFeed["items"][itens]["comment_count"]!=0:
                if userFeed["items"][itens]["caption"]!=None:
                    if datetime.fromtimestamp(userFeed["items"][itens]["caption"]["created_at"])>=query_start_collect:
                        #print(userFeed["items"][itens]["pk"])
                        #print(userFeed["items"][itens]["comment_count"])
                        #print(userFeed["items"][itens]["has_more_comments"])
                        #print(userFeed["items"][itens]["caption"]["text"])
                        #print(datetime.fromtimestamp(userFeed["items"][itens]["caption"]["created_at"]))
                        #print(userFeed["items"][itens]["caption"]["media_id"])
                        insert_comments(str(userFeed["items"][itens]["caption"]["media_id"]))
                    else:
                        more_available = False
    
        if more_available:
            more_available = userFeed.get('more_available', False)
            if userFeed.get('more_available', False)==True:
                max_id = userFeed["next_max_id"]   
                api.getUserFeed(usernameId=usernameId,maxid=max_id)
                userFeed = api.LastJson
                time.sleep(2)

In [34]:
def get_comments_by_hashTags(hashTags):
    more_available = True
    max_id = ''
    hTags = {}

    for tags in range(0,len(hashTags)):
        api.getHashtagFeed(hashtagString=hashTags[tags],maxid=max_id)
        hTags = api.LastJson

        while more_available:
            for itens in range(0,len(hTags['items'])):
                if hTags['items'][itens]["comment_count"]!=0:
                    if hTags["items"][itens]["caption"]!=None:
                        if datetime.fromtimestamp(hTags["items"][itens]["caption"]["created_at"])>=query_start_collect:
                            insert_comments(str(hTags["items"][itens]["caption"]["media_id"]))
                            print("HarshTag '" + hashTags[tags] + "': " +str(hTags["items"][itens]["caption"]["media_id"]))
                        else:
                            more_available = False

            if more_available:   
                more_available = hTags.get('more_available', False)
                if hTags.get('more_available', False)==True:
                    max_id = hTags["next_max_id"]   
                    api.getHashtagFeed(hashtagString=hashTags[tags],maxid=max_id)
                    hTags = api.LastJson
                    time.sleep(2)

In [38]:
def get_comments_by_location(listLocations):
    
    more_available = True
    max_id = ''
    place = {}

    for index in range(0,len(listLocations)):
        api.getLocationFeed(locationId=listLocations[index],maxid=max_id)
        place = api.LastJson

        while more_available:
            for itens in range(0,len(place['items'])):
                if place['items'][itens]["comment_count"]!=0:
                    if place["items"][itens]["caption"]!=None:
                        if datetime.fromtimestamp(place["items"][itens]["caption"]["created_at"])>=query_start_collect:
                            insert_comments(str(place["items"][itens]["caption"]["media_id"]))
                            print("Location '" + listLocations[index] + "': " +str(place["items"][itens]["caption"]["media_id"]))
                        else:
                            more_available = False

            if more_available:   
                more_available = place.get('more_available', False)
                if place.get('more_available', False)==True:
                    max_id = place["next_max_id"]   
                    api.getLocationFeed(locationId=listLocations[index],maxid=max_id)
                    place = api.LastJson
                    time.sleep(2)

In [33]:
def insert_comments(media_id):
    has_more_comments = True
    max_id = ''
    comments = []
    comm={}

    while has_more_comments:
        api.getMediaComments(media_id, max_id=max_id)
        comm = api.LastJson

        for itens in range(0,comm['comment_count']):
            if datetime.fromtimestamp(comm["comments"][itens]["created_at"])>=query_time_ini and datetime.fromtimestamp(comm["comments"][itens]["created_at"])<=query_time_end:
                if str(comm["comments"][itens]["user_id"])!=usernameId:
                    try:            
                        db_reacao.add_reacao_insta(1,4,comm["comments"][itens]["text"],0,comm["comments"][itens]["created_at"],str(comm["comments"][itens]["pk"]))
                    except Exception as e:
                        print("Erro inclusão comentário".center(110,"*"))
                        print("Identificador: " + str(comm["comments"][itens]["pk"]))
                        print("Comentário: " + str(comm["comments"][itens]["text"]))
                        print("User id: " + str(usernameId))
                        print("Media ID: " + str(media_id))
                        print("Erro: " + str(e))

        has_more_comments = comm.get('has_more_comments', False)
        if comm.get('has_more_comments', False)==True:
            max_id = comm.get('next_max_id', '')
            time.sleep(2)

# INICIO DA COLETA

In [ ]:
get_comments_by_usernameId(usernameId)
get_comments_by_hashTags(listHashTags)
get_comments_by_location(listLocations)

In [ ]:
api.getSelfUserFeed()  # get self user feed
print(api.LastJson) 

In [16]:
api.getLocationFeed(locationId="1008118425")
print(api.LastJson.keys()) 
print(api.LastJson["items"][0].keys()) 

dict_keys(['items', 'num_results', 'more_available', 'auto_load_more_enabled', 'media_count', 'location', 'status'])
dict_keys(['taken_at', 'pk', 'id', 'device_timestamp', 'media_type', 'code', 'client_cache_key', 'filter_type', 'image_versions2', 'original_width', 'original_height', 'location', 'lat', 'lng', 'user', 'can_viewer_reshare', 'caption', 'caption_is_edited', 'like_count', 'has_liked', 'comment_likes_enabled', 'comment_threading_enabled', 'has_more_comments', 'max_num_visible_preview_comments', 'preview_comments', 'comment_count', 'photo_of_you', 'can_viewer_save', 'organic_tracking_token'])


In [ ]:
api.getHashtagFeed(hashtagString="casariobotequim")
print(api.LastJson["items"][0].keys())
print(api.LastJson["items"][0]["caption"].keys())
print(api.LastJson["items"][0]["caption"]["media_id"])
print(api.LastJson["items"][0]["caption"]["created_at"])

In [ ]:
api.getHashtagFeed(hashtagString="casariobotequim")
print(api.LastJson.keys()) 
print(api.LastJson["num_results"]) 
print(api.LastJson["more_available"]) 
print(api.LastJson["items"]) 

In [ ]:
#pega dados do user id informado

api.getUsernameInfo(usernameId)
print(api.LastJson) 

In [ ]:
totalUserFeed = {}
api.getTotalUserFeed(usernameId=usernameId)
totalUserFeed = api.LastJson

In [ ]:
totalUserFeed.keys()
totalUserFeed["num_results"]

In [15]:
media_id = "1701881931908390505"
count = 100

max_id = ''
comments = []

api.getMediaComments(media_id, max_id=max_id)

True

In [14]:
    media_id = "1706576776958097375"
    has_more_comments = True
    max_id = ''
    comments = []
    comm={}

    while has_more_comments:
        time.sleep(1)
        api.getMediaComments(media_id, max_id=max_id)
        comm = api.LastJson

        for item_comm in range(0,len(comm["comments"])):
            if datetime.fromtimestamp(comm["comments"][item_comm]["created_at"])>=query_time_ini and datetime.fromtimestamp(comm["comments"][item_comm]["created_at"])<=query_time_end:
                if str(comm["comments"][item_comm]["user_id"])!=usernameId:
                    try:            
                        print(comm["comments"][item_comm]["text"])
                    except Exception as e:
                        print("Erro inclusão comentário".center(110,"*"))
                        print("Origem: " + origem)
                        print("Identificador: " + str(comm["comments"][item_comm]["pk"]))
                        print("Comentário: " + str(comm["comments"][item_comm]["text"]))
                        print("User id: " + str(usernameId))
                        print("Media ID: " + str(media_id))
                        print("Erro: " + str(e))

        has_more_comments = comm.get('has_more_comments', False)
        if comm.get('has_more_comments', False)==True:
            max_id = comm.get('next_max_id', '')
            time.sleep(2)

Que horas que ???
@josue_bernardo25 A Feijoada é servida 13h e o Samba começa às 15h
Coisa linda heim pai 👏👏👏👏
@casariobotequim esse domingo de carnaval vai ter a feijoada com samba?


In [22]:
len(api.LastJson["comments"])

20

In [21]:
for i in range(0,api.LastJson["comment_count"]):
    print(i)
    print(api.LastJson["comments"][i]["text"])

0
Lindos 😍
1
Boa Mbappé
2
@tonon_gabriel ai pqp ueueueuejsususjhehdhdhdhehhdhdhdhehehheheheuehueheheuehhehehehehehuehuheheheheheehehuehehe
3
@rafaelqbarbosa pare kkkkkkkkkkkkkkkkkkkkk
4
Noixxx 🤙🏽
5
Casa Rio mora no meu coração.
6
@lohanezm voce mora no nosso coração.
7
@rafaelqbarbosa vamos suspender o álcool, vamos? 😂
8
@lohanezm ok
9
@lohanezm affe
10
Mim DEICHE em PAS
11
Iiiiiii Perdemos um soldado.... felicidades 👏👏👏👏👏👏
12
@brendaaltoe é minha irmã, véi kkkkk
13
Nooooix
14
@brendaaltoe 😂😂😂😂😂😂
15
@rafaelqbarbosa como eu iria saber? Tipo... ela é linda demais pra ser sua irmã 😂.... Como isso é possível
16
@tety.lopes prazer irmã nunca mencionada 🙈
17
@brendaaltoe era de se esperar! Huhauaha
18
@tonon_gabriel né 😂
19
@brendaaltoe 😂😂😂😂😂😂😂😂😂 olaaaar
20


IndexError: list index out of range

In [8]:
print(api.LastJson)

{'comment_likes_enabled': True, 'comments': [{'pk': 17895405745159927, 'user_id': 5660856738, 'text': 'Que horas que ???', 'type': 0, 'created_at': 1517668391, 'created_at_utc': 1517668391, 'content_type': 'comment', 'status': 'Active', 'bit_flags': 0, 'user': {'pk': 5660856738, 'username': 'josue_bernardo25', 'full_name': 'Josue Bernardo', 'is_private': False, 'profile_pic_url': 'https://instagram.fbhz1-1.fna.fbcdn.net/vp/72de52466c4c5f2c080902d609efb028/5B676952/t51.2885-19/s150x150/26395849_163194964321222_8956283620037754880_n.jpg', 'profile_pic_id': '1694711840934626665_5660856738', 'is_verified': False}, 'did_report_as_spam': False, 'has_liked_comment': False, 'comment_like_count': 1, 'inline_composer_display_condition': 'never'}, {'pk': 17909674813100216, 'user_id': 190350270, 'text': '@josue_bernardo25 A Feijoada é servida 13h e o Samba começa às 15h', 'type': 0, 'created_at': 1517670036, 'created_at_utc': 1517670036, 'content_type': 'comment', 'status': 'Active', 'bit_flags': 